In [1]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

In [2]:
rawData = np.loadtxt('final_cleveland - no_title.csv' , delimiter=',')
rawData

array([[63.,  1.,  1., ...,  0.,  6.,  0.],
       [67.,  1.,  4., ...,  3.,  3.,  2.],
       [67.,  1.,  4., ...,  2.,  7.,  1.],
       ...,
       [68.,  1.,  4., ...,  2.,  7.,  2.],
       [57.,  1.,  4., ...,  1.,  7.,  3.],
       [57.,  0.,  2., ...,  1.,  3.,  1.]])

In [3]:
rawInputs = rawData[ : , 1:-1 ] #از ستون 1 تا یکی مونده به اخر
rawTargets = rawData[ : , -1]#
rawTargets = np.where(rawTargets == 0, 0,
                      np.where(rawTargets == 1, 1 ,
                        np.where(rawTargets == 2, 1 ,
                         np.where(rawTargets == 3, 1 ,
                          np.where(rawTargets == 4, 1 ,
                              rawTargets)))))
rawTargets

array([0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
       0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
       1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1.,
       1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
       1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
       1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 1.

In [4]:
numberOfOneTargets = int(np.sum(rawTargets))
numberOfZeroTargets = int( rawTargets.shape[0] - np.sum(rawTargets))
print( numberOfOneTargets , numberOfZeroTargets )

137 160


In [5]:
indexesToRemove = []
numberOfZeroTargets = 0

for i in range(rawTargets.shape[0]):
    if ( rawTargets[i] == 0):
        numberOfZeroTargets +=1
        if( numberOfZeroTargets > numberOfOneTargets):
            indexesToRemove.append(i)
            
print( 'zeros are : %s and we should remove %s indexes to balancing dataset'
      %(numberOfZeroTargets,len(indexesToRemove)))

zeros are : 160 and we should remove 23 indexes to balancing dataset


In [6]:
# rawBalancedInputs = np.delete( rawInputs, indexesToRemove, axis=0)
# rawBalancedTargets = np.delete( rawTargets , indexesToRemove , axis = 0 )

In [7]:
scaledInputs = preprocessing.scale(rawInputs)

In [8]:
shuffledIndex = np.arange(scaledInputs.shape[0])

np.random.shuffle(shuffledIndex)
scaledInputs

shuffleInputs = scaledInputs[shuffledIndex]
shuffleTargets = rawTargets[shuffledIndex]

In [9]:
samplesCount = shuffleInputs.shape[0]

trainCount = int( 0.8 * samplesCount )
validationCount = int( 0.1 * samplesCount )
testCount = int( samplesCount - (trainCount + validationCount) )

print( 'sample count is %s and train count is %s and validation count is %s and test count is %s'
      % ( samplesCount, trainCount, validationCount, testCount) )

sample count is 297 and train count is 237 and validation count is 29 and test count is 31


In [10]:
trainInputs = shuffleInputs[:trainCount]
trainTargets = shuffleTargets[:trainCount]

In [11]:
validationInputs = shuffleInputs[ trainCount : trainCount + validationCount ]
validationTargets = shuffleTargets[ trainCount : trainCount + validationCount ]

In [12]:
testInputs = shuffleInputs[ trainCount + validationCount : ]
testTargets = shuffleTargets[ trainCount + validationCount : ]

In [13]:
print( type(trainInputs) ,  type(trainTargets),
      type(validationInputs), type(validationTargets),
      type(testInputs), type(testTargets)
     )

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [14]:
trainInputs = trainInputs.astype(float)
trainTargets = trainTargets.astype(int)

validationInputs = validationInputs.astype(float)
validationTargets = validationTargets.astype(int)

testInputs = testInputs.astype(float)
testTargets = testTargets.astype(int)

In [15]:
inputsSize = 13
outputSize = 2
hiddenLayer = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense( hiddenLayer , activation='relu' ),
    tf.keras.layers.Dense( hiddenLayer , activation='relu' ),
    tf.keras.layers.Dense( outputSize , activation='softmax' )
])

In [16]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
epochSize = 100
batch_size =64
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)


model.fit( trainInputs ,
           trainTargets ,
           batch_size=batch_size,
           epochs=epochSize ,
           validation_data=(validationInputs, validationTargets) ,
           verbose=2,
           callbacks=[early_stopping]
         )

Epoch 1/100
4/4 - 1s - loss: 0.7894 - accuracy: 0.4557 - val_loss: 0.7449 - val_accuracy: 0.4138 - 921ms/epoch - 230ms/step
Epoch 2/100
4/4 - 0s - loss: 0.7161 - accuracy: 0.4937 - val_loss: 0.6812 - val_accuracy: 0.5172 - 35ms/epoch - 9ms/step
Epoch 3/100
4/4 - 0s - loss: 0.6559 - accuracy: 0.5907 - val_loss: 0.6324 - val_accuracy: 0.5517 - 34ms/epoch - 8ms/step
Epoch 4/100
4/4 - 0s - loss: 0.6070 - accuracy: 0.7131 - val_loss: 0.5911 - val_accuracy: 0.7586 - 33ms/epoch - 8ms/step
Epoch 5/100
4/4 - 0s - loss: 0.5645 - accuracy: 0.7932 - val_loss: 0.5543 - val_accuracy: 0.7931 - 34ms/epoch - 9ms/step
Epoch 6/100
4/4 - 0s - loss: 0.5242 - accuracy: 0.8354 - val_loss: 0.5218 - val_accuracy: 0.7931 - 36ms/epoch - 9ms/step
Epoch 7/100
4/4 - 0s - loss: 0.4879 - accuracy: 0.8270 - val_loss: 0.4926 - val_accuracy: 0.7931 - 34ms/epoch - 9ms/step
Epoch 8/100
4/4 - 0s - loss: 0.4549 - accuracy: 0.8439 - val_loss: 0.4683 - val_accuracy: 0.7931 - 33ms/epoch - 8ms/step
Epoch 9/100
4/4 - 0s - loss: 

In [18]:
testLoss , testAccuracy = model.evaluate(testInputs,testTargets)
print('Test loss of our model is: {0:.2f}. And our Test accuracy is: {1:.2f}%'.format(testLoss, testAccuracy*100.))

1/1 [==============================] - 0s 56ms/step - loss: 0.3255 - accuracy: 0.8387
Test loss of our model is: 0.33. And our Test accuracy is: 83.87%
